In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os

In [2]:
DATA_DIR = "./data"
REAL_DIR = os.path.join(DATA_DIR, "real")
FAKE_DIR = os.path.join(DATA_DIR, "fake")

In [ ]:
import numpy as np
import cv2
from scipy.fftpack import fft2, fftshift
from scipy.stats import linregress
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.color import rgb2gray
from skimage.util import img_as_float
from skimage.filters import gabor

def compute_fft_slope(image):
    """Compute the slope of the log-log power spectrum of the image."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = np.abs(fshift)**2
    radial_profile = radial_average(magnitude_spectrum)

    freqs = np.arange(1, len(radial_profile)+1)
    log_freqs = np.log(freqs)
    log_power = np.log(radial_profile + 1e-8)
    slope, *_ = linregress(log_freqs, log_power)
    return slope

def radial_average(power_spectrum):
    """Compute the radial average of a 2D power spectrum."""
    y, x = np.indices(power_spectrum.shape)
    center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int32)
    tbin = np.bincount(r.ravel(), power_spectrum.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / (nr + 1e-8)
    return radialprofile

def compute_noise_residual(image):
    """Compute residual noise statistics after denoising."""
    gray = rgb2gray(image / 255.0)
    sigma_est = np.mean(estimate_sigma(gray, channel_axis=None))
    denoised = denoise_nl_means(gray, h=1.15 * sigma_est, fast_mode=True)
    residual = gray - denoised
    return np.mean(residual), np.std(residual)

def compute_dct_entropy(image):
    """Compute entropy of DCT coefficients over non-overlapping 8x8 blocks."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    gray = gray[:h - h % 8, :w - w % 8]
    blocks = gray.reshape(h // 8, 8, w // 8, 8).swapaxes(1, 2).reshape(-1, 8, 8)
    dct_blocks = np.array([cv2.dct(block.astype(np.float32)) for block in blocks])
    coeffs = dct_blocks.reshape(-1)
    hist, _ = np.histogram(coeffs, bins=256, range=(-50, 50), density=True)
    entropy = -np.sum(hist * np.log(hist + 1e-8))
    return entropy

def compute_fractal_dimension(image):
    """Estimate the fractal dimension via box counting method."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)
        return len(np.where(S > 0)[0])

    Z = (gray > gray.mean()).astype(np.uint8)
    p = min(Z.shape)
    n = 2**np.floor(np.log2(p)).astype(int)
    sizes = 2**np.arange(np.log2(n), 1, -1).astype(int)
    counts = [boxcount(Z, size) for size in sizes]
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

def compute_phase_coherence(image):
    """Estimate average phase coherence across orientations."""
    gray = rgb2gray(image / 255.0)
    theta_vals = np.linspace(0, np.pi, 8, endpoint=False)
    coherences = []
    for theta in theta_vals:
        real, imag = gabor(gray, frequency=0.2, theta=theta)
        magnitude = np.sqrt(real**2 + imag**2) + 1e-8
        coherence = (real / magnitude).mean()
        coherences.append(coherence)
    return np.mean(coherences)

# Wrapper function to extract all features from image + path
def extract_features(image_path):
    image = cv2.imread(image_path)
    features = {
        "fft_slope": compute_fft_slope(image),
        "residual_mean": compute_noise_residual(image)[0],
        "residual_std": compute_noise_residual(image)[1],
        "dct_entropy": compute_dct_entropy(image),
        "fractal_dimension": compute_fractal_dimension(image),
        "phase_coherence": compute_phase_coherence(image),
    }
    return features


ModuleNotFoundError: No module named 'jpegio'